# Aula 4: Pandas II

Ler e manipular dados é sem dúvida o ponto fonte do Pandas, e maior parte do trabalho de qualquer Analista de Dados. Para isso precisamos de dados. No dia a dia o mais comum é consumir dados que estão armazenados em csv e arquivos de excel. Outra fonte comum de dados são os bancos SQL.

Durante essa aula vamos utilizar algumas tabelas de dados no formato separado por vírgula (csv) disponíveis gratuitamente na internet. Essas tabelas ficam disponíveis no [site](https://people.sc.fsu.edu/~jburkardt/data/csv/csv.html) da Universidade Estadual da Flórica (FSU):

* https://people.sc.fsu.edu/~jburkardt/data/csv/csv.html

Existem diversas fontes de dados para praticar. O [Kaggle](https://www.kaggle.com/) é uma das mais interessantes pois além de trazer dados e problemas reais e com uma comunidade que se ajuda e ensina.

```
Não é exagero dizer que para muitos Cientistas de Dados, de 80 a 90% do tempo deva ser gasto carregando, entendendo e limpando tabelas.
```

## Lendos os Dados

Uma das funções mais importantes de leitura do pandas é o `pd.read_csv()`. Você pode conferir a documentação dela [aqui](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#). Das diversar formas de ler um arquivo a mais comum talvez seja informar o caminho até o arquivo mas existem inúmeras formas. Se ela estiver armazenada em uma página da web, basta informar o endereço como _string_.

Vamos fazer isso com a tabela dos times de baseball de 2012 (MLB):

In [14]:
!pip install nb_black --quiet
%load_ext nb_black

import pandas as pd
import numpy as np

times_mlb = pd.read_csv(
    "https://people.sc.fsu.edu/~jburkardt/data/csv/mlb_teams_2012.csv"
)

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

A função `pd.read_csv()` lê um arquivo separado por vírgulas e devolve uma variável do tipo `pd.DataFrame`. Depois de ler uma tabela é muito comum chamar o método `.head()` que imprime as primeiras linhas da tabela (5 primeiras se nada for informado).

In [12]:
times_mlb.head()

,Team,"""Payroll (millions)""","""Wins"""
0,Nationals,81.34,98
1,Reds,82.20,97
2,Yankees,197.96,95
3,Giants,117.62,94
4,Braves,83.31,94


<IPython.core.display.Javascript object>

Outra prática comum é checar as últimas linhas. O método `tail()` da cabo disso. A seguir mostro como você pode informar o número de linhas impressos:

In [4]:
times_mlb.tail(6)

,Team,"""Payroll (millions)""","""Wins"""
24,Red Sox,173.18,69
25,Indians,78.43,68
26,Twins,94.08,66
27,Rockies,78.06,64
28,Cubs,88.19,61
29,Astros,60.65,55


<IPython.core.display.Javascript object>

Imprimir as primeiras e últimas linhas de uma tabela é uma ótima forma de:

* i. garantir que os dados foram lidos corretamente;
* ii. observar inconsistências nos dados.

Outras práticas comuns são:
* i. conferir o tamanho da tabela usando o atributo `.shape`;
* ii. examinar os tipos dos dados com `.dtypes`.

In [5]:
times_mlb.shape

(30, 3)

<IPython.core.display.Javascript object>

A tabela tem 30 linhas e 3 colunas. E quais os tipos das variáveis?

In [13]:
times_mlb.dtypes

Team                     object
 "Payroll (millions)"    object
 "Wins"                   int64
dtype: object

<IPython.core.display.Javascript object>

`object` se assemelha à `string`.

Um padrão comum na nomenclatura das colunas é utilizar somente letras minúscula e _underline_. O mesmo padrão com letras maiúsculas também é bastante comum. Abaixo eu defino uma função para limpar os nomes e faço a readequação por meio de uma _list comprehension_.

In [15]:
def limpa_nome(nome):
    return (
        nome.lower()
        .strip()
        .replace('"', "")
        .replace("(", "")
        .replace(")", "")
        .replace(" ", "_")
    )


times_mlb.columns = [limpa_nome(coluna) for coluna in times_mlb.columns]

times_mlb.head(2)

,team,payroll_millions,wins
0,Nationals,81.34,98
1,Reds,82.20,97


<IPython.core.display.Javascript object>

Um problema muito comum enfrentado a ler uma tabela é a confusão entre os sinais utilizados para casas decimais. Vou simular um problema desse tipo utilizando o método `.astype()`.

In [8]:
times_mlb["payroll_millions"] = (
    times_mlb["payroll_millions"].astype(str).str.replace(".", ",", regex=False)
)
times_mlb.head(2)

,team,payroll_millions,wins
0,Nationals,"81,34",98
1,Reds,"82,2",97


<IPython.core.display.Javascript object>

In [ ]:
times_mlb.iloc[:, 1] = times_mlb.iloc[:, 1].fillna(-9999)

times_mlb.loc["payroll_millions"] = times_mlb.loc["payroll_millions"].fillna(-99999)

In [9]:
times_mlb.dtypes

team                object
payroll_millions    object
wins                 int64
dtype: object

<IPython.core.display.Javascript object>

Aqui utilizamos fizemos as seguintes operações:

* 1) Convertemos o tipo da variável para _string_ com `astype(str)`:
* 2) Acessamos os métodos _string_ chamando _.str_;
* 3) Trocamos todas os pontos por víruglas `.replace(".", ",", regex=False)`. `regex=False` evita que o ponto seja interpretado como uma expressão regular.

Isso serviu para mostrar como seria uma tabela que usava vírgulas ao invés de pontos para apontar casas decimais. Agora que temos uma coluna "mal-comportada" podemos mostrar como corrigir essa situação. Uma forma de prevenir esse comportamento seria ler a tabela utilizando `pd.read_csv(<caminho do arquivo>, decimal=",")`, onde `decimal=","` informa o caractére empregado para separar casas decimais.

Outra forma seria combibar `.str.replace().astype()` para transformar a coluna.

In [10]:
times_mlb["payroll_millions"] = (
    times_mlb["payroll_millions"].str.replace(",", ".", regex=False).astype(float)
)
times_mlb.dtypes

team                 object
payroll_millions    float64
wins                  int64
dtype: object

<IPython.core.display.Javascript object>

## Operações com DataFrames

Uma série de operações estão disponíveis. Vamos examinar algumas delas.


`.mean()` retorna a média:

In [16]:
times_mlb.mean()

payroll_millions    98.019
wins                81.000
dtype: float64

<IPython.core.display.Javascript object>

In [18]:
times_mlb.mean(axis=1)

0      89.670
1      89.600
2     146.480
3     105.810
4      88.655
5      74.685
6     106.755
7      87.215
8      77.085
9     121.745
10    110.150
11     99.150
12     90.570
13     90.960
14     90.325
15    127.770
16     77.640
17     71.215
18     65.620
19     78.485
20     83.675
21     74.240
22     66.455
23     93.535
24    121.090
25     73.215
26     80.040
27     71.030
28     74.595
29     57.825
dtype: float64

<IPython.core.display.Javascript object>

É possível selecionar apenas uma coluna da seguinte forma:

In [ ]:
times_mlb["wins"].mean()

Ou várias ao mesmo tempo:

In [ ]:
times_mlb[["payroll_millions", "wins"]].mean()

Analogamente, `.sum()` retorna a soma:

In [ ]:
times_mlb["payroll_millions"].sum()

In [19]:
times_mlb["team"].unique()

array(['Nationals', 'Reds', 'Yankees', 'Giants', 'Braves', 'Athletics',
       'Rangers', 'Orioles', 'Rays', 'Angels', 'Tigers', 'Cardinals',
       'Dodgers', 'White Sox', 'Brewers', 'Phillies', 'Diamondbacks',
       'Pirates', 'Padres', 'Mariners', 'Mets', 'Blue Jays', 'Royals',
       'Marlins', 'Red Sox', 'Indians', 'Twins', 'Rockies', 'Cubs',
       'Astros'], dtype=object)

<IPython.core.display.Javascript object>

In [20]:
times_mlb["team"].nunique()

30

<IPython.core.display.Javascript object>

In [21]:
tabela_2 = times_mlb.iloc[:5].copy()
tabela_2

,team,payroll_millions,wins
0,Nationals,81.34,98
1,Reds,82.20,97
2,Yankees,197.96,95
3,Giants,117.62,94
4,Braves,83.31,94


<IPython.core.display.Javascript object>

In [22]:
tabela_suja = pd.concat([times_mlb, tabela_2], ignore_index=True)
tabela_suja.shape

(35, 3)

<IPython.core.display.Javascript object>

In [24]:
tabela_suja.head(2)

,team,payroll_millions,wins
0,Nationals,81.34,98
1,Reds,82.20,97


<IPython.core.display.Javascript object>

In [27]:
tabela_suja.loc[tabela_suja.duplicated()]

,team,payroll_millions,wins
30,Nationals,81.34,98
31,Reds,82.20,97
32,Yankees,197.96,95
33,Giants,117.62,94
34,Braves,83.31,94


<IPython.core.display.Javascript object>

In [28]:
tabela_suja.loc[tabela_suja.duplicated(keep=False)]

,team,payroll_millions,wins
0,Nationals,81.34,98
1,Reds,82.20,97
2,Yankees,197.96,95
3,Giants,117.62,94
4,Braves,83.31,94
30,Nationals,81.34,98
31,Reds,82.20,97
32,Yankees,197.96,95
33,Giants,117.62,94
34,Braves,83.31,94


<IPython.core.display.Javascript object>

In [29]:
tabela_suja.drop_duplicates().shape

(30, 3)

<IPython.core.display.Javascript object>

In [30]:
tabela_suja.iloc[:5, 1] = np.nan

<IPython.core.display.Javascript object>

In [31]:
tabela_suja.head()

,team,payroll_millions,wins
0,Nationals,NaN,98
1,Reds,NaN,97
2,Yankees,NaN,95
3,Giants,NaN,94
4,Braves,NaN,94


<IPython.core.display.Javascript object>

In [32]:
tabela_suja.isnull().sum()

team                0
payroll_millions    5
wins                0
dtype: int64

<IPython.core.display.Javascript object>

In [40]:
tabela_suja.iloc[10:14, 1] = np.nan

<IPython.core.display.Javascript object>

fillna()
bfill()
ffill()

In [44]:
tabela_suja["payroll_millions"]  # .ffill()

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5      55.37
6     120.51
7      81.43
8      64.17
9     154.49
10       NaN
11       NaN
12       NaN
13       NaN
14     97.65
15    174.54
16     74.28
17     63.43
18     55.24
19     81.97
20     93.35
21     75.48
22     60.91
23    118.07
24    173.18
25     78.43
26     94.08
27     78.06
28     88.19
29     60.65
30     81.34
31     82.20
32    197.96
33    117.62
34     83.31
Name: payroll_millions, dtype: float64

<IPython.core.display.Javascript object>

In [45]:
tabela_suja["payroll_millions"] = tabela_suja["payroll_millions"].fillna(
    tabela_suja["payroll_millions"].mean()
)

<IPython.core.display.Javascript object>

In [46]:
tabela_suja["payroll_millions"].value_counts()

96.381154     9
83.310000     1
173.180000    1
81.430000     1
94.080000     1
197.960000    1
97.650000     1
120.510000    1
81.340000     1
74.280000     1
88.190000     1
60.650000     1
75.480000     1
60.910000     1
64.170000     1
82.200000     1
63.430000     1
117.620000    1
118.070000    1
78.060000     1
93.350000     1
81.970000     1
55.240000     1
154.490000    1
174.540000    1
55.370000     1
78.430000     1
Name: payroll_millions, dtype: int64

<IPython.core.display.Javascript object>

In [47]:
jogadores_mlb = pd.read_csv(
    "https://people.sc.fsu.edu/~jburkardt/data/csv/mlb_players.csv"
)

ParserError: Error tokenizing data. C error: EOF inside string starting at row 1035

<IPython.core.display.Javascript object>

In [56]:
jogadores_mlb = pd.read_csv(
    "https://people.sc.fsu.edu/~jburkardt/data/csv/mlb_players.csv",
    engine="python",
    error_bad_lines=False,
)

jogadores_mlb.head()

Skipping line 1036: unexpected end of data


,Name,"""Team""","""Position""","""Height(inches)""","""Weight(lbs)""","""Age"""
0,Adam Donachie,"""BAL""","""Catcher""",74,180,22.99
1,Paul Bako,"""BAL""","""Catcher""",74,215,34.69
2,Ramon Hernandez,"""BAL""","""Catcher""",72,210,30.78
3,Kevin Millar,"""BAL""","""First Baseman""",72,210,35.43
4,Chris Gomez,"""BAL""","""First Baseman""",73,188,35.71


<IPython.core.display.Javascript object>

In [ ]:
mask_duplicados = jogadores_mlb.duplicated()
mask_duplicados

In [ ]:
mask_duplicados.sum()

In [ ]:
mask_duplicados_nome = jogadores_mlb["Name"].duplicated()
mask_duplicados_nome

In [ ]:
mask_duplicados_nome.sum()

In [ ]:
mask_duplicados_nome = jogadores_mlb.duplicated(subset=["Name"], keep=False)
mask_duplicados_nome.sum()

In [ ]:
jogadores_mlb.loc[mask_duplicados_nome]

In [ ]:
mask_duplicados_nome = jogadores_mlb.duplicated(subset=["Name"], keep="last")
mask_duplicados_nome.sum()

jogadores_mlb_2 = jogadores_mlb.loc[~mask_duplicados_nome]
jogadores_mlb_2

In [ ]:
data_cadastro = {
    "ID" : [1,2,3,5,12,1,5,6,7],
    "Saldo" : [10,-20,30,20,120,110,50,60,70]
}


df_cadastro = pd.DataFrame(data_cadastro)
df_cadastro

In [ ]:
df_cadastro.drop_duplicates(subset=["ID"], keep="first")

In [ ]:
df_cadastro.drop_duplicates(subset=["ID"], keep=False)

In [ ]:
df_cadastro = (df_cadastro
               .sort_values(by=["Saldo", "ID"], ascending=[True, False])
               .drop_duplicates(subset=["ID"], keep="last")
              )

In [ ]:
df_cadastro.sort_values(by=1,axis=1)

In [ ]:
jogadores_mlb.loc[jogadores_mlb["Name"].str.startswith('Tony Pe')]

In [ ]:
jogadores_mlb.shape

In [ ]:
jogadores_mlb.tail()

In [57]:
jogadores_mlb.columns = [limpa_nome(coluna) for coluna in jogadores_mlb.columns]

jogadores_mlb.head(2)

,name,team,position,heightinches,weightlbs,age
0,Adam Donachie,"""BAL""","""Catcher""",74,180,22.99
1,Paul Bako,"""BAL""","""Catcher""",74,215,34.69


<IPython.core.display.Javascript object>

In [58]:
def limpa_texto(texto):
    return texto.strip().replace('"', "")


for coluna in jogadores_mlb.columns:

    if jogadores_mlb[coluna].dtype == "object":
        jogadores_mlb[coluna] = jogadores_mlb[coluna].apply(lambda x: limpa_texto(x))

jogadores_mlb.head()

,name,team,position,heightinches,weightlbs,age
0,Adam Donachie,BAL,Catcher,74,180,22.99
1,Paul Bako,BAL,Catcher,74,215,34.69
2,Ramon Hernandez,BAL,Catcher,72,210,30.78
3,Kevin Millar,BAL,First Baseman,72,210,35.43
4,Chris Gomez,BAL,First Baseman,73,188,35.71


<IPython.core.display.Javascript object>

In [84]:
jogadores_mlb

,name,team,position,heightinches,weightlbs,age
0,Adam Donachie,BAL,Catcher,74,180,22.99
1,Paul Bako,BAL,Catcher,74,215,34.69
2,Ramon Hernandez,BAL,Catcher,72,210,30.78
3,Kevin Millar,BAL,First Baseman,72,210,35.43
4,Chris Gomez,BAL,First Baseman,73,188,35.71
...,...,...,...,...,...,...
1029,Brad Thompson,STL,Relief Pitcher,73,190,25.08
1030,Tyler Johnson,STL,Relief Pitcher,74,180,25.73
1031,Chris Narveson,STL,Relief Pitcher,75,205,25.19
1032,Randy Keisler,STL,Relief Pitcher,75,190,31.01


<IPython.core.display.Javascript object>

In [90]:
endereco_dados = "https://gist.githubusercontent.com/purp/cb71552e6313a832ec3c/raw/48204f9a270e5e12e2105e7ab82859de8800b24c/schedule_urls.csv"

mlb_ids = pd.read_csv(endereco_dados, header=None, names=["team_name", "team_id", "_"],)
mlb_ids.head()

,team_name,team_id,_
0,Arizona Diamondbacks,ari,webcal://mlb.am/tix/diamondbacks_schedule_full
1,Atlanta Braves,atl,webcal://mlb.am/tix/braves_schedule_full
2,Baltimore Orioles,bal,webcal://mlb.am/tix/orioles_schedule_full
3,Boston Red Sox,bos,webcal://mlb.am/tix/redsox_schedule_full
4,Chicago White Sox,cws,webcal://mlb.am/tix/whitesox_schedule_full


<IPython.core.display.Javascript object>

In [92]:
# del mlb_ids["_"]
mlb_ids.drop(columns=["_"], inplace=True)

mlb_ids.head()

,team_name,team_id
0,Arizona Diamondbacks,ari
1,Atlanta Braves,atl
2,Baltimore Orioles,bal
3,Boston Red Sox,bos
4,Chicago White Sox,cws


<IPython.core.display.Javascript object>

In [93]:
mlb_ids["team_id"] = mlb_ids["team_id"].str.upper()
mlb_ids.head()

,team_name,team_id
0,Arizona Diamondbacks,ARI
1,Atlanta Braves,ATL
2,Baltimore Orioles,BAL
3,Boston Red Sox,BOS
4,Chicago White Sox,CWS


<IPython.core.display.Javascript object>

In [ ]:
pd.merge([
    jogadores_mlb, # left 
    mlb_ids # right
])

In [99]:
df_nomes = jogadores_mlb.loc[:5, ["name", "team"]]

df_nomes.merge(
    mlb_ids, left_on=["team"], right_on=["team_id"],  # how=""
)

,name,team,team_name,team_id
0,Adam Donachie,BAL,Baltimore Orioles,BAL
1,Paul Bako,BAL,Baltimore Orioles,BAL
2,Ramon Hernandez,BAL,Baltimore Orioles,BAL
3,Kevin Millar,BAL,Baltimore Orioles,BAL
4,Chris Gomez,BAL,Baltimore Orioles,BAL
...,...,...,...,...
934,Brad Thompson,STL,St. Louis Cardinals,STL
935,Tyler Johnson,STL,St. Louis Cardinals,STL
936,Chris Narveson,STL,St. Louis Cardinals,STL
937,Randy Keisler,STL,St. Louis Cardinals,STL


<IPython.core.display.Javascript object>

In [111]:
df_nomes2 = jogadores_mlb.loc[:5, ["name", "team"]]
df_nomes2

,name,team
0,Adam Donachie,BAL
1,Paul Bako,BAL
2,Ramon Hernandez,BAL
3,Kevin Millar,BAL
4,Chris Gomez,BAL
5,Brian Roberts,BAL


<IPython.core.display.Javascript object>

In [102]:
mlb_ids["team_id"].duplicated().sum()

0

<IPython.core.display.Javascript object>

In [103]:
df_nomes2.merge(mlb_ids, left_on=["team"], right_on=["team_id"], how="left")

,name,team,team_name,team_id
0,Adam Donachie,BAL,Baltimore Orioles,BAL
1,Paul Bako,BAL,Baltimore Orioles,BAL
2,Ramon Hernandez,BAL,Baltimore Orioles,BAL
3,Kevin Millar,BAL,Baltimore Orioles,BAL
4,Chris Gomez,BAL,Baltimore Orioles,BAL
5,Brian Roberts,BAL,Baltimore Orioles,BAL


<IPython.core.display.Javascript object>

In [104]:
df_nomes2.merge(mlb_ids, left_on=["team"], right_on=["team_id"], how="right")

,name,team,team_name,team_id
0,NaN,NaN,Arizona Diamondbacks,ARI
1,NaN,NaN,Atlanta Braves,ATL
2,Adam Donachie,BAL,Baltimore Orioles,BAL
3,Paul Bako,BAL,Baltimore Orioles,BAL
4,Ramon Hernandez,BAL,Baltimore Orioles,BAL
5,Kevin Millar,BAL,Baltimore Orioles,BAL
6,Chris Gomez,BAL,Baltimore Orioles,BAL
7,Brian Roberts,BAL,Baltimore Orioles,BAL
8,NaN,NaN,Boston Red Sox,BOS
9,NaN,NaN,Chicago White Sox,CWS


<IPython.core.display.Javascript object>

In [105]:
df_nomes2.merge(mlb_ids, left_on=["team"], right_on=["team_id"], how="inner")

,name,team,team_name,team_id
0,Adam Donachie,BAL,Baltimore Orioles,BAL
1,Paul Bako,BAL,Baltimore Orioles,BAL
2,Ramon Hernandez,BAL,Baltimore Orioles,BAL
3,Kevin Millar,BAL,Baltimore Orioles,BAL
4,Chris Gomez,BAL,Baltimore Orioles,BAL
5,Brian Roberts,BAL,Baltimore Orioles,BAL


<IPython.core.display.Javascript object>

In [106]:
df_nomes2.merge(mlb_ids, left_on=["team"], right_on=["team_id"], how="outer")

,name,team,team_name,team_id
0,Adam Donachie,BAL,Baltimore Orioles,BAL
1,Paul Bako,BAL,Baltimore Orioles,BAL
2,Ramon Hernandez,BAL,Baltimore Orioles,BAL
3,Kevin Millar,BAL,Baltimore Orioles,BAL
4,Chris Gomez,BAL,Baltimore Orioles,BAL
5,Brian Roberts,BAL,Baltimore Orioles,BAL
6,NaN,NaN,Arizona Diamondbacks,ARI
7,NaN,NaN,Atlanta Braves,ATL
8,NaN,NaN,Boston Red Sox,BOS
9,NaN,NaN,Chicago White Sox,CWS


<IPython.core.display.Javascript object>

In [107]:
df_nomes3 = df_nomes2.copy()
df_nomes3["team"] = "AD(UASDJ)"
df_nomes3

,name,team
0,Adam Donachie,AD(UASDJ)
1,Paul Bako,AD(UASDJ)
2,Ramon Hernandez,AD(UASDJ)
3,Kevin Millar,AD(UASDJ)
4,Chris Gomez,AD(UASDJ)
5,Brian Roberts,AD(UASDJ)


<IPython.core.display.Javascript object>

In [108]:
df_nomes3.merge(mlb_ids, left_on=["team"], right_on=["team_id"], how="outer")

,name,team,team_name,team_id
0,Adam Donachie,AD(UASDJ),NaN,NaN
1,Paul Bako,AD(UASDJ),NaN,NaN
2,Ramon Hernandez,AD(UASDJ),NaN,NaN
3,Kevin Millar,AD(UASDJ),NaN,NaN
4,Chris Gomez,AD(UASDJ),NaN,NaN
5,Brian Roberts,AD(UASDJ),NaN,NaN
6,NaN,NaN,Arizona Diamondbacks,ARI
7,NaN,NaN,Atlanta Braves,ATL
8,NaN,NaN,Baltimore Orioles,BAL
9,NaN,NaN,Boston Red Sox,BOS


<IPython.core.display.Javascript object>

In [114]:
df_nomes2.merge(
    mlb_ids, left_on=["team"], right_on=["team_id"], how="inner", validate="many_to_one"
)

,name,team,team_name,team_id
0,Adam Donachie,BAL,Baltimore Orioles,BAL
1,Paul Bako,BAL,Baltimore Orioles,BAL
2,Ramon Hernandez,BAL,Baltimore Orioles,BAL
3,Kevin Millar,BAL,Baltimore Orioles,BAL
4,Chris Gomez,BAL,Baltimore Orioles,BAL
5,Brian Roberts,BAL,Baltimore Orioles,BAL


<IPython.core.display.Javascript object>

In [ ]:
times_mlb['team'].isin(mlb_ids['team']).mean()

In [ ]:
mlb_ids['team']

In [ ]:
mlb_ids['team'].apply(lambda x : " ".join(x.split(" ")[1:]))

In [ ]:
times_mlb['team'].isin(mlb_ids['team'].apply(lambda x : " ".join(x.split(" ")[1:]))).mean()

In [ ]:
times_mlb.loc[~times_mlb['team'].isin(mlb_ids['team'].apply(lambda x : " ".join(x.split(" ")[1:])))]

In [ ]:
ids_1 = mlb_ids.copy()
ids_1["team"] = ids_1["team"].apply(lambda x : " ".join(x.split(" ")[1:]))


ids_2 = ids_1.copy()
mask_dois_nomes = ids_2["team"].str.count(" ") >= 1
ids_2 = ids_2.loc[mask_dois_nomes]
ids_2["team"] = ids_2["team"].apply(lambda x : " ".join(x.split(" ")[1:]))

mlb_ids_abreviados = pd.concat([ids_1, ids_2], ignore_index=True)

In [ ]:
times_mlb['team'].isin(mlb_ids_abreviados['team']).mean()

In [129]:
df1 = pd.DataFrame([[1, "a"], [-2, "z",]], columns=["number", "letter"])


df2 = pd.DataFrame([[3, "x"], [-4, "d"]], columns=["number", "letter"])
df2

,number,letter
0,3,x
1,-4,d


<IPython.core.display.Javascript object>

In [82]:
df1

,letter,number
0,a,1
1,b,2


<IPython.core.display.Javascript object>

In [83]:
pd.concat([df1, df2], axis=0, ignore_index=True)

,letter,number,y
0,a,1,NaN
1,b,2,NaN
2,NaN,3,c
3,NaN,4,d


<IPython.core.display.Javascript object>

In [131]:
pd.concat([df1, df2], axis=0, ignore_index=True, sort=False)

,number,letter
0,1,a
1,-2,z
2,3,x
3,-4,d


<IPython.core.display.Javascript object>

In [133]:
pd.concat([df1, df2], axis=0, ignore_index=True, sort=True)

,letter,number
0,a,1
1,z,-2
2,x,3
3,d,-4


<IPython.core.display.Javascript object>

In [66]:
pd.concat([df1, df2], axis=1)

,letter,number,letter,number
0,a,1,c,3
1,b,2,d,4


<IPython.core.display.Javascript object>